In [1]:
import datetime
import csv
import time
import pandas as pd

In [2]:
def import_dataset(is_cleaned):
    # load dataset as pd dataframe
    if not is_cleaned:
        missing_values = ['Dimensions unavailable']
        dframe = pd.read_csv('museum_data.csv', low_memory=False, na_values=missing_values)

        # rename column names for easier calling and set object_id as index
        dframe.columns = dframe.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
        dframe.set_index('object_id')

        # remove duplicates
        len_before = len(dframe)
        dframe.drop_duplicates(['object_number'], inplace=True)
        print('Removed {} duplicates \n'.format(len_before - len(dframe)))
    else:
        dframe = pd.read_csv('museum_data_cleaned.csv', low_memory=False)
    return dframe

# Create a Readme document

Prints out a Pandas Dataframe, which is used the create a Readme doc for the dataset.
Every attribute/column is displayed, with name, dtype and a short description if necessary will be added manually. This document can be used for future usage and to get a fast understanding of the dataset.

In [8]:
df = import_dataset(False)
frames = []
counter = 1
for column in df:
    types = df[column].apply(type).value_counts(dropna=True)
    str_nr, int_nr, bool_nr = 0, 0, 0

    for i in range(0, len(types.values)):
        if types.index[i].__name__ == 'str':
            str_nr = types.values[i]
        elif types.index[i].__name__ == 'int':
            int_nr = types.values[i]
        elif types.index[i].__name__ == 'bool':
            bool_nr = types.values[i]

        is_dtype = ''
        if int_nr != 0:
            is_dtype = 'int'
        elif str_nr != 0:
            is_dtype = 'String'
        elif bool_nr != 0:
            is_dtype = 'boolean'

    new_row = pd.DataFrame({'attribute': [column + ':'],
                            'dtype': [is_dtype],
                            'column': [counter],
                            'Description': ['.']})
    frames.append(new_row)
    counter += 1

result = pd.concat(frames)
result.set_index('column', inplace=True)
print(result.head(100))

Removed 2743 duplicates 

                       attribute    dtype Description
column                                               
1                    unnamed:_0:      int           .
2                 object_number:   String           .
3                  is_highlight:  boolean           .
4              is_public_domain:  boolean           .
5                     object_id:      int           .
6                    department:   String           .
7                   object_name:   String           .
8                         title:   String           .
9                       culture:   String           .
10                       period:   String           .
11                      dynasty:   String           .
12                        reign:   String           .
13                    portfolio:   String           .
14                  artist_role:   String           .
15                artist_prefix:   String           .
16          artist_display_name:   String           .
17

In [15]:
def transform_artist_dates(df):
    """
    The artist dates are transformed to have a uniform format.
    Multiple dates are put in a list of strings and in the format:
    '%Y-%m-%d', with a minus in front if the date is B.C.
    """
    for col in ['artist_begin_date', 'artist_end_date']:
        print(col)
        for index, row in df.iterrows():
            row = row[col]
            transformed_date_list = []
            if type(row) is str:
                row = row.replace(' ', '')
                if '|' in row:
                    date_list = row.split('|')
                else:
                    date_list = [row]

                for d in date_list:
                    if '-' in d and len(d) == 10:
                        dt_date = d
                    elif len(d) <= 5:
                        dt_date = d + '-01-01'
                    transformed_date_list.append(dt_date)

                # save transformed back to the dataframe
                df.at[index, col] = transformed_date_list

In [14]:
def all_col_with_multiple_values(df):
    """
    Returns the names of all columns which have the '|' symbol in their cells, which indicates multiple values in on cell.
    """
    col_list = []
    for col in df:
        has_divider = False
        for cell in df[col]:
            if type(cell) is str and '|' in cell:
                has_divider = True
        if has_divider:
            col_list.append(col)

    return col_list

In [16]:
def split_attr_in_lists(df):
    """
    All values from one cell are put in a list, for easier access when analysing the data. Data is in one string and
    simply divided by '|', so before the analysis, every value has to be separated. Every column where there are cells
    with multiple values, they are transformed to lists.
    Format: ['example', 'example']
    """
    for col in all_col_with_multiple_values(df):
        print(col)
        for index, row in df.iterrows():
            row = row[col]
            if type(row) is str:
                if '|' in row:
                    new_list = row.split('|')
                else:
                    new_list = [row]

                # save list back to the dataframe
                df.at[index, col] = new_list

Run all data cleaning operations on the dataframe and save it as a pickle, for easy usage at a later time.

In [17]:
df = import_dataset(False)

transform_artist_dates(df)
split_attr_in_lists(df)
df.to_pickle('museum_data_cleaned.pkl')
print('Dataframe successfully cleaned.')

Removed 2743 duplicates 

artist_begin_date
artist_end_date
title
artist_role
artist_prefix
artist_display_name
artist_display_bio
artist_suffix
artist_alpha_sort
artist_nationality
medium
geography_type
city
state
county
country
region
subregion
locale
locus
excavation
classification
tags
Dataframe successfully cleaned.
